<a href="https://colab.research.google.com/github/wikiban43/Misinformation-Analysis/blob/main/community.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
import os
import json
import time
import pandas as pd
from datetime import datetime,timedelta
import time
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Community Analysis

In [119]:
from datetime import datetime
import ast

#HELPER METHODS TO GET THE DATE AND AUTHOR NAME FROM TEXT
def get_date(string):
  try:
    date = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S.%fZ')
  except:
    date = string
    return date
  #date = str(date.month)+"/"+str(date.year)
  return date.date()

def get_author_name(string):
  try:
    d = ast.literal_eval(string)
    return d["username"]
  except:
    return "None"
    
#METHOD TO GET RETWEET NETWORK FROM ANY DF
def get_network(df):
  username_l = list(df["author.username"].unique())
  #Get the retweeted users
  if "text" in df.columns:
    df["retweeted user"] = df["text"].apply(lambda x: get_retweeted_user(x))

  #Get all retweet edges using Soham's data (A,B would mean A retweets B)
  #edges = []
  df['edges'] = list(zip(df["author.username"], df["retweeted user"]))
  #Unique edges (unweighted graph)
  edges = df['edges']

  #Get only the within group edges for network analysis
  relevant_edges = []
  for edge in edges:
    if edge[0] in username_l and edge[1] in username_l:
      relevant_edges.append(edge)
  edges = relevant_edges

  #Weighted graph if needed
  #weight_dict = get_edges_with_weights(df['edges'].to_list())
  #dump_csv_for_community_detection(weight_dict)

  #Remove duplicate edges for unweighted graph
  edges = set(edges)
  #print("Number of unique edges: "+str(len(edges)))
  #print("\n")
  if len(edges) == 0:
    print("No retweet edges found!")
    return None
  #Get the RT network between core users (one edge only between a pair of users even if there are multiple RTs between them)
  # generate generic network graph instance
  nx_graph = nx.DiGraph()
  for edge in edges:#[0:1000]:
    #print(edges_dict[edge])
    try:
      nx_graph.add_edge(edge[0],edge[1])
    except:
      continue
  return nx_graph,len(edges)
def get_retweeted_user(text):
    text_l = text.split(" ")
    if text_l[0]=="RT":
      target_uname = text_l[1].replace("@","")
      target_uname = target_uname.rstrip(":")
      #print("Target:"+target_uname)
      return target_uname

#RETURNS 1 IF THE TWEET IS A RETWEET, ELSE 0
def get_retweet_flag(text):
  text_l = text.split(" ")
  flag = 0
  if text_l[0]=="RT":
    flag = 1
  return flag   

# Get the frequency, PDF and CDF for each value in the series
def get_cdf_data(df,col_name):
  # Frequency
  try:
    df = df.sort_values(col_name).reset_index()
  except:
    print("Dataframe already sorted, probably!")
  stats_df = df.groupby(col_name)[col_name].agg('count').pipe(pd.DataFrame).rename(columns = {col_name: 'frequency'})
  # PDF
  stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
  # CDF
  stats_df['cdf'] = stats_df['pdf'].cumsum()
  stats_df = stats_df.reset_index()
  stats_df
  return stats_df

In [110]:
#GET MEDIA AND INFLUENCERS FROM THE DISMIS DATASET

df_dismis = pd.read_excel("drive/My Drive/MSR/icwsm_analysis/influencers_final.xlsx")
df_medinf = df_dismis.loc[(df_dismis["sub_category"]=='media_house') | (df_dismis["sub_category"]=='platform_first')].reset_index()

df_dismis["sub_category"].unique()

#Politicians list
df_politicians = pd.read_csv('drive/My Drive/MSR/icwsm_analysis/politicians.csv')
pol_l = list(df_politicians['screen_name'].unique())
#Media and influencer list
media_inf_l = list(df_medinf["username"].unique())

In [111]:
#READ DATA FOR THE EVENTS

path = "drive/My Drive/MSR/icwsm_datasets/"
df = pd.read_csv(path+"caa_nrc.csv")
#df = pd.read_csv(path+"budget2022_jan_jul.csv")
#df = pd.read_csv(path+"budget2022_6months.csv")


#Some preprocessing for budget data
path = "drive/My Drive/MSR/icwsm_datasets/old datasets/"
df1 = pd.read_csv(path+'budget2022_jan_mar.csv')
df2 = pd.read_csv(path+'budget2022_mar_jul.csv')
df = pd.concat([df1,df2])

df.shape  

(846306, 11)

In [112]:
df["author.username"] = df["author"].apply(lambda x: get_author_name(x))  #For Budget only

In [113]:
df.columns

Index(['id', 'text', 'created_at', 'author_id', 'in_reply_to_user_id', 'lang',
       'author', 'entities', 'referenced_tweets', 'media', 'geo',
       'author.username'],
      dtype='object')

In [114]:
#Divide the df into partitions based on date (budget)

df_part1 = df.query("created_at >= '2022-01-01' and created_at < '2022-02-01'")
df_part2 = df.query("created_at >= '2022-01-01' and created_at < '2022-03-01'")
df_part3 = df.query("created_at >= '2022-01-01' and created_at < '2022-04-01'")
df_part4 = df.query("created_at >= '2022-01-01' and created_at < '2022-05-01'")
df_part5 = df.query("created_at >= '2022-01-01' and created_at < '2022-06-01'")
df_part6 = df.query("created_at >= '2022-01-01' and created_at < '2022-07-01'")
'''
#Divide the df into partitions based on date (CAA)
df_part1 = df.query("created_at >= '2019-12-01' and created_at < '2020-01-01'")
df_part2 = df.query("created_at >= '2019-12-01' and created_at < '2020-02-01'")
df_part3 = df.query("created_at >= '2019-12-01' and created_at < '2020-03-01'")
df_part4 = df.query("created_at >= '2019-12-01' and created_at < '2020-04-01'")
'''

df_list = [df_part1,df_part2,df_part3,df_part4,df_part5,df_part6]

In [121]:
#FIRST, GET THE HIGH RETWEETING USERS AND THE HIGHLY RETWEETED USERS
from matplotlib import pyplot as plt
import networkx as nx
import networkx.algorithms.community as nxcom
import networkx as nx

for df in df_list:
  print("#################################################################")
  df["is_retweet"] = df["text"].apply(lambda x: get_retweet_flag(x))
  df_temp_g = df.groupby("author.username")["is_retweet"].sum().reset_index()
  df_temp_g = df_temp_g.sort_values("is_retweet",ascending=False).reset_index()
  stats_df1 = get_cdf_data(df_temp_g,"is_retweet")

  retweets_made_cutoff = int(stats_df1.loc[stats_df1["cdf"]>=0.997].head(1)["is_retweet"]) #Based on knee point
  high_retweeting_users = list(df_temp_g.loc[df_temp_g["is_retweet"]>=retweets_made_cutoff]["author.username"].unique())
  #print(high_retweeting_users)
  #Get retweet count received by each user (without any duplicate filtering)
  try:
    df_rt = df.groupby(["author.username"])["public_metrics.retweet_count"].sum().reset_index()
    df_rt = df_rt.sort_values(by=["public_metrics.retweet_count"],ascending=False).reset_index()
    stats_df2 = get_cdf_data(df_rt,"public_metrics.retweet_count")
    retweets_received_cutoff = int(stats_df2.loc[stats_df2["cdf"]>=0.8].head(1)["public_metrics.retweet_count"])
    #SELECT USERS WITH HIGH RETWEET ATTRACTION FROM THE DATAFRAME, BASED ON THE CDF
    df_rt_high_retweets = df_rt.loc[df_rt["public_metrics.retweet_count"]>retweets_received_cutoff].reset_index()
    high_retweet_users = list(set(list(df_rt_high_retweets["author.username"].unique())))
    #print(high_retweet_users)
  except:
    print("Probably retweet count public metrics was not found.")
  



  #df with media and influencers
  #df_champs = df.loc[df["author.username"].isin(media_inf_l)].reset_index()

  #df with high retweet receivers and media and influencers
  #df_champs = df.loc[(df["author.username"].isin(high_retweet_users))|(df["author.username"].isin(media_inf_l))].reset_index()

  #df with high retweet receivers, politicians, and media and influencers
  #df_champs = df.loc[(df["author.username"].isin(high_retweet_users))|(df["author.username"].isin(media_inf_l))|(df["author.username"].isin(pol_l))].reset_index()

  #df with politicians, and media and influencers
  #df_champs = df.loc[(df["author.username"].isin(media_inf_l))|(df["author.username"].isin(pol_l))].reset_index()

  #df with high retweet receivers, high retweeters, politicians, and media and influencers
  #df_champs = df.loc[(df["author.username"].isin(high_retweet_users))|(df["author.username"].isin(high_retweeting_users))|(df["author.username"].isin(media_inf_l))|(df["author.username"].isin(pol_l))].reset_index()

  #df with high retweet receivers, pols, and media and influencers
  df_champs = df.loc[(df["author.username"].isin(high_retweeting_users))|(df["author.username"].isin(media_inf_l))|(df["author.username"].isin(pol_l))].reset_index()


  #print("Media and influencers:"+str(len(media_inf_l)))
  #print("Highly retweeted users:"+str(len(high_retweet_users)))
  #print("Final dataset with both has users:"+str(df_champs["author.username"].nunique()))
  df_champs.shape
  #df_champs.shape
  #df_champs.drop_duplicates().shape
  #df_champs = df_champs[["id","text","created_at","author.username","public_metrics.retweet_count","referenced_tweets"]].drop_duplicates()
  #df_champs = df_champs[["id","text","created_at","author.username","public_metrics.retweet_count","referenced_tweets"]].drop_duplicates()
  df_champs = df_champs[["id","text","created_at","author.username","referenced_tweets"]].drop_duplicates()
  #df_media = df_medinf2.loc[df_medinf2["sub_category"]=="media_house"].reset_index() #Enable if only media tweets are needed
  #df_inf = df_medinf2.loc[df_medinf2["sub_category"]=="platform_first"].reset_index()#Enable if only influencer tweets are needed
  df = df_champs
  #SOME PREPROCESSING ON THE DF TO GET THE DATES IN STRING FORMAT ETC
  df = df.sort_values(by="created_at")#.reset_index()
  df["created_at"] = df["created_at"].astype(str)
  df.reset_index(drop=True,inplace=True)
  #GET THE RT NETWORK FOR THIS EVENT

  #pos = nx.spring_layout(nx_graph, k=0.1)
  #plt.rcParams.update({'figure.figsize': (15, 10)})
  #nx.draw_networkx(nx_graph,pos=pos,node_size=0,edge_color="#444444",alpha=0.05,with_labels=False)
  nx_graph,_ = get_network(df)


  #nx.write_gexf(nx_graph, "drive/My Drive/MSR/icwsm_analysis/Budget_pols_retweeters_medinf.gexf", version="1.2draft")
  #GET COMMUNITIES USING THE GREEDY MODULARITY TECHNIQUE FOR THE RETWEET GRAPH FOR THIS EVENT
  communities = sorted(nxcom.greedy_modularity_communities(nx_graph), key=len, reverse=True)
  #communities = sorted(community.louvain_community(nx_graph))
  #pickle.dump(communities,open('drive/My Drive/MSR/icwsm_analysis/Budget_communities_medinf_pol.pickle','wb'))
  #caa_comm = pickle.load(open('drive/My Drive/MSR/icwsm_analysis/CAA_communities.pickle','rb'))
  df.loc[df["author.username"]=='BhootSantosh']
  caa_comm = communities
  
  #FOR EACH COMMUNITY, FIND THE CENTRAL/MOST INFLUENTIAL USER
  leader_l = []
  num_edges_l = []
  for i in range(0,len(caa_comm)):
    iterator=iter(caa_comm[i])
    comm = list(iterator)
    df_comm = df.loc[df["author.username"].isin(comm)].reset_index()
    comm_G,num_edges = get_network(df_comm)
    influence_dict = nx.pagerank_scipy(comm_G)
    leader = max(influence_dict, key=influence_dict.get)
    print("The leader of community-"+str(i)+" is:"+leader+" with PageRank "+str(influence_dict[leader]))
    print("The community has "+str(len(comm_G.edges()))+" edges.")
    leader_l.append(leader)
    num_edges_l.append(num_edges)
  print('\n\n')
  #for username in iterator:
  #  print(username)
  df_comm = pd.DataFrame()
  df_comm["leader"] = leader_l
  df_comm["num_edges"] = num_edges_l

#################################################################
Probably retweet count public metrics was not found.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: DeprecationWarning:

networkx.pagerank_scipy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.



The leader of community-0 is:sanghaviharsh with PageRank 0.14804889852332598
The community has 51 edges.
The leader of community-1 is:CNBCTV18News with PageRank 0.17508518883420734
The community has 96 edges.
The leader of community-2 is:PIB_India with PageRank 0.21518292922389673
The community has 41 edges.
The leader of community-3 is:ETNOWlive with PageRank 0.37363403151286434
The community has 47 edges.
The leader of community-4 is:TheDeshBhakt with PageRank 0.22259286662532513
The community has 39 edges.
The leader of community-5 is:BiIndia with PageRank 0.10061882067170756
The community has 22 edges.
The leader of community-6 is:editorji with PageRank 0.18424690030555135
The community has 16 edges.
The leader of community-7 is:ABPNews with PageRank 0.5197067869611879
The community has 18 edges.
The leader of community-8 is:KTRTRS with PageRank 0.4471298487476566
The community has 15 edges.
The leader of community-9 is:vaibhavpawarmp with PageRank 0.30287648635117803
The community

In [ ]:
import plotly.express as px
fig = px.line(stats_df1,x="is_retweet",y="cdf")#,x="author.username")
fig.show()
df["created_at"] = df["created_at"].apply(lambda x: get_date(x))

In [107]:
df_comm.to_csv("drive/My Drive/MSR/icwsm_analysis/Budget_communities.csv")

In [108]:
df_comm.head()

,leader,num_edges
0,priyankagandhi,5494
1,PIB_India,3832
2,Dev_Fadnavis,827
3,mkstalin,359
4,Outlookindia,237


In [ ]:
 #VISUALIZE GRAPH BASED ON COMMUNITIES (NOT IMMEDIATELY NEEDED)
 
 def set_node_community(G, communities):
  '''Add community to node attributes'''
  for c, v_c in enumerate(communities):
      for v in v_c:
          # Add 1 to save 0 for external edges
          G.nodes[v]['community'] = c + 1
def set_edge_community(G):
  '''Find internal edges and add their community to their attributes'''
  for v, w, in G.edges:
      if G.nodes[v]['community'] == G.nodes[w]['community']:
          # Internal edge, mark with community
          G.edges[v, w]['community'] = G.nodes[v]['community']
      else:
          # External edge, mark as 0
          G.edges[v, w]['community'] = 0
def get_color(i, r_off=1, g_off=1, b_off=1):
  '''Assign a color to a vertex.'''
  r0, g0, b0 = 0, 0, 0
  n = 16
  low, high = 0.1, 0.9
  span = high - low
  r = low + span * (((i + r_off) * 3) % n) / (n - 1)
  g = low + span * (((i + g_off) * 5) % n) / (n - 1)
  b = low + span * (((i + b_off) * 7) % n) / (n - 1)
  return (r, g, b) 

G_social = nx_graph

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
plt.style.use('dark_background')
# Set node and edge communities
set_node_community(G_social, communities)
set_edge_community(G_social)
# Set community color for internal edges
external = [(v, w) for v, w in G_social.edges if G_social.edges[v, w]['community'] == 0]
internal = [(v, w) for v, w in G_social.edges if G_social.edges[v, w]['community'] > 0]
internal_color = ["black" for e in internal]
node_color = [get_color(G_social.nodes[v]['community']) for v in G_social.nodes]
# external edges
nx.draw_networkx(
    G_social,
    pos=pos,
    node_size=0,
    edgelist=external,
    edge_color="silver",
    node_color=node_color,
    alpha=0.2,
    with_labels=False)
# internal edges
nx.draw_networkx(
    G_social, pos=pos,
    edgelist=internal,
    edge_color=internal_color,
    node_color=node_color,
    alpha=0.05,
    with_labels=False)